In [1]:
import requests
import re
import urllib
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from random import randint
from datetime import datetime

In [2]:
def get_url(title, location, page):
    template_url = 'https://www.indeed.com/jobs?q={}&l={}&fromage=14&start={}'
    url = template_url.format(urllib.parse.quote(title), urllib.parse.quote(location), page)
    return url

In [3]:
def get_records(card):

    try:
        job_title = card.find('h2', {'class':'jobTitle'}).findAll('span')[-1].text
    except:
        job_title = ''
    
    try:
        job_conpany = card.find('span', {'class':'companyName'}).text
    except:
        job_conpany = ''

    try:
        job_rating = card.find('span', {'class':'ratingsDisplay withRatingLink'}).text
    except:
        job_rating = ''
    
    try:
        job_location = [i.text for i in card.find_all('div', {'class':'companyLocation'})]
    except:
        job_location = ''
    
    try:
        job_salary =  card.find('span', {'class':'salary-snippet'}).text
    except:
        job_salary = ''
    
    try:
        job_date = card.find('span', {'class':'date'}).text
    except:
        job_date = ''

    extract_date = datetime.today().strftime('%Y-%m-%d')

    job_url = 'https://www.indeed.com'+ card['href']
    return job_title, job_conpany, job_rating, job_location, job_salary, job_date, extract_date, job_url


In [4]:
def get_detail(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    #job detail
    info = {}
    try:
        job_detail = soup.find_all('div', {'class':'jobsearch-JobDescriptionSection-sectionItem'})
        for detail in job_detail:
            if detail.find('span'):
                info[detail.find_all('div')[0].text] = [i.text for i in detail.find_all('span')]
            else:
                info[detail.find_all('div')[0].text] = [i.text for i in detail.find_all('div')[1:]]
    except:
        pass
    
    #job qualification
    try:
        jobQual = soup.find('div', {'id':'qualificationsSection'})
        title = jobQual.find('h2').text
        info[title] = [i.text for i in jobQual.find_all('li', {'class':'icl-u-xs-p--none jobsearch-ReqAndQualSection-item'})]
    except:
        pass

    #job description
    try:
        jobdesc = soup.find('div', {'id':'jobDescriptionText'})
        info['Description'] = jobdesc.format_string
    except:
        pass
        
    res.close()
    return info



In [5]:
def get_info(title, location):
    header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36"}
    records = []
    details = []
    no_of_page = 0
    for i in range(0, 600, 10):
        url = get_url(title, location, i)
        print(url)
        res = requests.get(url, header)
        soup = BeautifulSoup(res.text, 'html.parser')

        #get total jobs
        if no_of_page == 0:
            try:
                no_of_page_string = soup.find(id='searchCountPages').text.strip()
                no_of_page = int(re.sub('\W+', '', no_of_page_string.split()[3]))
            except:
                print('No total number')
                pass

        try:
            cards = soup.find(id='mosaic-provider-jobcards').findChildren("a" , recursive=False)
        except:
            print('No job list')
            break

        for card in cards:
            record = get_records(card)
            records.append(record)
            wait = randint(5, 12)
            sleep(wait)
            detail = get_detail(record[-1])
            details.append(detail)

        wait = randint(30, 60)
        sleep(wait)

        # res.close()
    return records, details, no_of_page*15

In [6]:
def saveDate(title, records, details):
    col = ['Title', 'Company', 'Rating', 'Location', 'Salary', 'Post_date', 'Extract_date', 'Job_url']
    df_basic = pd.DataFrame(records)
    df_basic.columns = col
    df_detail = pd.DataFrame.from_dict(details)
    df = pd.concat([df_basic, df_detail], axis=1)
    df.to_csv(f'{title}.csv')

In [13]:
no_of_jobs = {}

#get records from 'Data scientist' in 'UniedStates'
# records, details, total = get_info('Data scientist', 'United states')
# try:
#     saveDate(f'Data scientist', records, details)
# except:
#     print("No data stored")
# no_of_jobs['Data scientist'] = total

# sleep(300)
# #get records from 'Software programmer' in 'UniedStates'
# records, details, total = get_info('Software programmer', 'United states')
# try:
#     saveDate(f'Software programmer', records, details)
# except:
#     print("No data stored")
# no_of_jobs['Software programmer'] = total

# sleep(300)
# #get records from 'Web programmer' in 'UniedStates'
records, details, total = get_info('Web programmer', 'United states')
try:
    saveDate(f'Web programmer', records, details)
except:
    print("No data stored")
no_of_jobs['Web programmer'] = total

# sleep(300)
# #get records from 'Information architect' in 'UniedStates'
# records, details, total = get_info('Information architect', 'United states')
# saveDate('Information architect', records, details)
# no_of_jobs['Information architect'] = total

https://www.indeed.com/jobs?q=Web%20programmer&l=United%20states&fromage=14&start=0
No total number
No job list
No data stored


In [11]:
no_of_jobs

{'Software programmer': 930780}

In [12]:
len(records)

889